In [119]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# Requests
import requests
from time import sleep
from tqdm import tqdm

# Para convertir de string a diccionario
import ast
import os
import json

import sys
sys.path.append("../")
import src.soporte_spotify as api

In [120]:
# sp = api.load_credentials()
# sp.current_user()

In [121]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv')
zara.sample()

,brand,username,user_id,playlists,playlist_ids,playlist_ids_limited,artistas
406,Zara,Kat Albuquerque ✨,qzkdkcjf9v356no2hjlr4p3zn,"{'Amei Te Ver': '1EBNjgh6qwt6digummbG0S', 'Pal...","['1EBNjgh6qwt6digummbG0S', '4rW5JrOFOfWeOuJvzW...","['1EBNjgh6qwt6digummbG0S', '4rW5JrOFOfWeOuJvzW...","{'4iWkwAVzssjb8XgxdoOL6M': 'TIAGO IORC', '2QLS..."


### Eliminar aquellos users cuyas playlists no tenían artistas (no había canciones)

In [122]:
zara = zara.loc[zara["artistas"] != "{}"]

### Reemplazar null por None para poder convertirlo a diccionarios

In [123]:
for idx in zara.index:
    zara.loc[idx,"artistas"] = zara.loc[idx,"artistas"].replace("null","None")

### Convertir:
- playlists: de STR a dict
- playlist_ids: de STR a list
- playlist_ids_limited: de STR a list
- artistas: de STR a dict

In [124]:
zara["playlists"] = zara["playlists"].apply(ast.literal_eval)
zara["playlist_ids"] = zara["playlist_ids"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
zara["playlist_ids_limited"] = zara["playlist_ids_limited"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
zara["artistas"] = zara["artistas"].apply(ast.literal_eval)
zara.to_csv("../datos/01 Spotify/01_followers_zara.csv",index=False)

### Ver cuantos artistas tenemos en total

In [125]:
suma = 0
for artista in zara["artistas"]:
    suma += len(artista)

suma

323870

# Diccionario de artistas únicos
- Creamos un diccionario de artistas únicos a partir de todos los obtenidos
- De forma que solo consultaremos los géneros una sola vez

In [126]:
artistas_unicos = {}
for dictio in zara["artistas"]:
    for id_artista, artista in dictio.items():
        if id_artista not in artistas_unicos:
            artistas_unicos[id_artista] = artista
# Eliminar la Key "None"
del artistas_unicos["None"]

In [127]:
len(artistas_unicos)

110684

Diccionario Nº Veces que aparece un artista
- de los usuarios, obtendremos el número de veces que sale un artista
- De esta manera sabremos la popularidad dentro de los seguidores de Zara


In [128]:
conteo_artistas = {}
for dictio in zara["artistas"]:
    for artista in dictio.values():
        conteo_artistas[artista] = conteo_artistas.get(artista,0) + 1

# Ordenar los artistas de mayor a menor repetición
conteo_artistas = sorted(conteo_artistas.items(), key=lambda x: x[1], reverse=True)
# Quitamos None, que es un artista vacío
conteo_artistas.pop(1)

(None, 328)

In [129]:
len(conteo_artistas)

108095

# Generar Nuevo DF con
- Marca
- Número de Usuarios
- diccionario artistas únicos
- ranking_artistas

In [130]:
df_zara = pd.DataFrame({
    "brand" : [zara["brand"].unique()[0]] ,
    "followers" : zara.shape[0],
    "unique_artists": str(artistas_unicos),
    "artist_ranking" : str(conteo_artistas)
})
df_zara["unique_artists"] = df_zara["unique_artists"].apply(ast.literal_eval)
df_zara["artist_ranking"] = df_zara["artist_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df_zara

,brand,followers,unique_artists,artist_ranking
0,Zara,848,"{'3qm84nBOXUEQ2vnTfUTTFC': 'Guns N' Roses', '6...","[(The Weeknd, 346), (Dua Lipa, 298), (Coldplay..."


In [131]:
df_zara.to_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv",index=False)